In [ ]:
# transformer with phi3 is only available in latest version
!pip uninstall -y transformers && pip install git+https://github.com/huggingface/transformers

Found existing installation: transformers 4.42.0.dev0
Uninstalling transformers-4.42.0.dev0:
  Successfully uninstalled transformers-4.42.0.dev0
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-qsx17r4i
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-qsx17r4i
  Resolved https://github.com/huggingface/transformers to commit 96eb06286b63c9c93334d507e632c175d6ba8b28
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.42.0.dev0-py3-none-any.whl size=9140346 sha256=29e166d3c4d14a853f2edb72dabab8e6c7218c7d0507f4f294efc21b32df5f1d
  Stored in directory: /tmp/pip-ephem-wheel-cache-lc937fyt/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers


In [ ]:
!pip install "pandas<2.0.0"

In [ ]:
!pip install accelerate

In [ ]:
!pip install datasets

##  Import Packages

In [ ]:
import pandas
import torch
import accelerate
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from datasets import Dataset, load_dataset, load_from_disk

In [ ]:
# Setting torch seed
torch.random.manual_seed(0)

## Defining model & tokenizers

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

In [ ]:
aspects = ["food", "service", "ambience", "price", "other"]
sentiments = ["positive", "neutral", "negative"]


In [ ]:
messages = [
    {
        "role": "system",
        "content": f"Your task is to analyse and perform aspect based sentiment analysis on the given restaurant reviews text. The aspects should only be {aspects} and sentiments should only be {sentiments}. The output should contain Aspects as keys and polarity of sentiment. Write 'None' as sentiment is aspect is not present in the input. Each aspects should only have one sentiment and not every aspect is necessarily present. Do not provide any further explanation.",
    },
    {
        "role": "user",
        "content": "Add to that great service and great food at a reasonable price and you have yourself the beginning of a great evening",
    },
    {
        "role": "assistant",
        "content": "{'ambience': None,'food':'positive','other': None,'price':'positive','service':'positive',}",
    },
    {
        "role": "user",
        "content": "I can understand the prices if it served better food, like some Chinese restaurants in midtown/uptown area.",
    },
]

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

output = pipe(messages, **generation_args)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:533: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [ ]:
print(output[0]["generated_text"])

 {'ambience': None,'food': 'negative','price': 'neutral','service': None,}


## Loading data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Loading test data
dataset = load_from_disk("/content/drive/MyDrive/Scientific_project/dataset/test_dataset.hf")
dataset[0]

{'text': 'bread top notch well',
 'category': ['food'],
 'polarity': ['positive'],
 'joint': ['food#positive'],
 'category_labels': [0, 1, 0, 0, 0],
 'polarity_labels': [0, 0, 1],
 'joint_labels': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'true_labels': {'ambience': None,
  'food': 'positive',
  'other': None,
  'price': None,
  'service': None}}

### Prompting

In [ ]:
aspects = ["food", "service", "ambience", "price", "other"]
sentiments = ["positive", "neutral", "negative"]


def prompting(data, prompt, prompt_id):
    result = pipe(
        [
            [
                {
                    "role": "system",
                    "content": f"{prompt}",
                },
                {
                    "role": "user",
                    "content": "Add to that great service and great food at a reasonable price and you have yourself the beginning of a great evening",
                },
                {
                    "role": "assistant",
                    "content": "{'service':'positive','food':'positive','price':'positive'}",
                },
                {"role": "user", "content": x},
            ]
            for x in data["text"]
        ],
        **generation_args,
    )
    output = [_res[0]["generated_text"] for _res in result]

    #   for o in output:
    #       try:
    #           _output = eval(o.strip())
    #           parsed_output.append(_output)
    #       except Exception as e:
    #           print(f"Output not a valid json. \nException: {e} \nOutput: {o}")
    #           parsed_output.append({})
    #       print("\n")
    return {prompt_id: output}

In [ ]:
prompt = f"Your task is to conduct aspect-based sentiment analysis on the provided restaurant reviews. Focus on identifying the following aspects: {aspects}. Classify sentiments for each aspect as either {sentiments}. Present the results in a dictionary format with each aspect and its respective sentiment. Ensure that each aspect is assigned only one sentiment, and note that not every aspect must be present in each review."
prompt_id = "prompt_1"


dataset1 = dataset.map(
    lambda _rows: prompting(_rows, prompt, prompt_id), batched=True, batch_size=100
)

Parameter 'function'=<function <lambda> at 0x7dff65d43c70> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/749 [00:00<?, ? examples/s]

In [ ]:
dataset1.to_pandas().to_csv(
    "/content/drive/MyDrive/Scientific_project/thesis/prompt_1.csv"
)

In [ ]:
prompt = f"You are a skilled data analyst. Your task is to extract and analyze sentiments from user reviews and structure the information into a JSON format. Analyze the provided user reviews and create a JSON summary. Focus on these key aspects: {aspects} and sentiments: {sentiments}."
prompt_id = "prompt_2"



dataset2 = dataset.map(
    lambda _rows: prompting(_rows, prompt, prompt_id), batched=True, batch_size=100
)

Map:   0%|          | 0/749 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:533: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
dataset2.to_pandas().to_csv(
    "/content/drive/MyDrive/Scientific_project/thesis/prompt2.csv"
)

In [ ]:
prompt = f"Your task is to perform aspect-based sentiment analysis on the following restaurant review texts. Focus specifically on the aspects: {aspects}. Each aspect should be assigned a sentiment from the options {sentiments}. Provide the output in a dictionary format where the keys are the aspects and the values are the corresponding sentiments. Each aspect should have only one sentiment, and it's not necessary for all aspects to be present in every review. Do not include any further explanation or additional text."
prompt_id = "prompt_3"


dataset_os3 = dataset.map(
    lambda _rows: prompting(_rows, prompt, prompt_id), batched=True, batch_size=100
)

Map:   0%|          | 0/749 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:533: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [ ]:
dataset_os3.to_pandas().to_csv(
    "/content/drive/MyDrive/Scientific_project/thesis/prompt_3.csv"
)